In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

file_path_2010 = Path("../Data/RawData/2010.csv")
file_path_2011 = Path("../Data/RawData/2011.csv")
file_path_2012 = Path("../Data/RawData/2012.csv")
file_path_2013 = Path("../Data/RawData/2013.csv")
file_path_2014 = Path("../Data/RawData/2014.csv")
file_path_2015 = Path("../Data/RawData/2015.csv")
file_path_2016 = Path("../Data/RawData/2016.csv")
file_path_2017 = Path("../Data/RawData/2017.csv")
file_path_2018 = Path("../Data/RawData/2018.csv")
file_path_2019 = Path("../Data/RawData/2019.csv")

data_2010 = pd.read_csv(file_path_2010)
data_2010['Year'] = '2010'
data_2011 = pd.read_csv(file_path_2011)
data_2011['Year'] = '2011'
data_2012 = pd.read_csv(file_path_2012)
data_2012['Year'] = '2012'
data_2013 = pd.read_csv(file_path_2013)
data_2013['Year'] = '2013'
data_2014 = pd.read_csv(file_path_2014)
data_2014['Year'] = '2014'
data_2015 = pd.read_csv(file_path_2015)
data_2015['Year'] = '2015'
data_2016 = pd.read_csv(file_path_2016)
data_2016['Year'] = '2016'
data_2017 = pd.read_csv(file_path_2017)
data_2017['Year'] = '2017'
data_2018 = pd.read_csv(file_path_2018)
data_2018['Year'] = '2018'
data_2019 = pd.read_csv(file_path_2019)
data_2019['Year'] = '2019'

data_2010 = data_2010[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2011 = data_2011[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2012 = data_2012[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2013 = data_2013[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2014 = data_2014[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2015 = data_2015[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2016 = data_2016[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2017 = data_2017[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2018 = data_2018[["Year","Player","Tm","Pos","FantasyPoints"]]
data_2019 = data_2019[["Year","Player","Tm","Pos","FantasyPoints"]]

data_2010 = data_2010.set_index("Year")
data_2011 = data_2011.set_index("Year")
data_2012 = data_2012.set_index("Year")
data_2013 = data_2013.set_index("Year")
data_2014 = data_2014.set_index("Year")
data_2015 = data_2015.set_index("Year")
data_2016 = data_2016.set_index("Year")
data_2017 = data_2017.set_index("Year")
data_2018 = data_2018.set_index("Year")
data_2019 = data_2019.set_index("Year")


full_data = pd.concat([data_2010, data_2011, data_2012, data_2013, data_2014, data_2015, data_2016, data_2017, data_2018, data_2019])

full_data = full_data[full_data['FantasyPoints'] != 0]
full_data = full_data[full_data['Pos'] != '0']

full_data.to_csv('../Data/CleanData/full_data.csv', encoding='utf-8')

# full_data.sort_values('Tm')
# full_data_player_sort.to_csv('../Data/CleanData/full_data_player_sort.csv', encoding='utf-8')

player_mean = full_data.groupby(['Player']).mean()

# player_mean.plot(kind='bar')
player_mean.hvplot.bar(width=800, rot=90)
# data_2010[["Player", "FantasyPoints"]].plot(kind='bar')

In [ ]:
team_mean = full_data.groupby(['Tm']).mean()
team_mean.hvplot.bar(width=800, rot=90)

In [ ]:
position_mean = full_data.groupby(['Pos']).mean()
position_mean.hvplot.bar(width=800, rot=90)